# 相似度度量深入分析
# Similarity Metrics Deep Dive

这个笔记本深入探讨各种相似度度量的数学原理和实际应用。

## 学习目标
- 理解不同相似度度量的数学原理
- 比较各种度量方法的特点和适用场景
- 可视化相似度计算结果

In [ ]:
# 导入必要的库
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from src.basic_vector_search import BasicVectorSearch
from src.utils import plot_similarity_heatmap

## 1. 数据准备

让我们创建一些测试向量来演示不同的相似度度量：

In [ ]:
# 创建测试向量
test_vectors = np.array([
    [1, 0, 0],      # 单位向量 X
    [0, 1, 0],      # 单位向量 Y
    [0, 0, 1],      # 单位向量 Z
    [1, 1, 0],      # 对角线向量
    [2, 0, 0],      # 2倍X向量
    [1, 1, 1],      # 三维对角线
    [-1, 0, 0],     # 负X向量
    [0.5, 0.5, 0.7] # 混合向量
])

# 向量标签
vector_labels = ['X', 'Y', 'Z', 'XY', '2X', 'XYZ', '-X', 'Mixed']

print("测试向量:")
for i, (vec, label) in enumerate(zip(test_vectors, vector_labels)):
    print(f"{label}: {vec}")

## 2. 余弦相似度 (Cosine Similarity)

余弦相似度测量两个向量之间的角度，范围从-1到1。

In [ ]:
# 计算余弦相似度矩阵
cosine_sim_matrix = cosine_similarity(test_vectors)

# 可视化余弦相似度矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(cosine_sim_matrix, 
            xticklabels=vector_labels, 
            yticklabels=vector_labels,
            annot=True, 
            cmap='RdYlBu_r', 
            center=0,
            fmt='.3f')
plt.title('余弦相似度矩阵')
plt.tight_layout()
plt.show()

# 分析关键观察
print("关键观察:")
print(f"X 和 2X 的余弦相似度: {cosine_sim_matrix[0, 4]:.3f} (完全相同方向)")
print(f"X 和 -X 的余弦相似度: {cosine_sim_matrix[0, 6]:.3f} (完全相反方向)")
print(f"X 和 Y 的余弦相似度: {cosine_sim_matrix[0, 1]:.3f} (垂直)")

## 3. 欧几里得距离 (Euclidean Distance)

欧几里得距离是最直观的距离度量，计算向量之间的直线距离。

In [ ]:
# 计算欧几里得距离矩阵
euclidean_dist_matrix = euclidean_distances(test_vectors)

# 可视化欧几里得距离矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(euclidean_dist_matrix, 
            xticklabels=vector_labels, 
            yticklabels=vector_labels,
            annot=True, 
            cmap='viridis_r',
            fmt='.3f')
plt.title('欧几里得距离矩阵')
plt.tight_layout()
plt.show()

print("关键观察:")
print(f"X 和 2X 的欧几里得距离: {euclidean_dist_matrix[0, 4]:.3f} (长度差异)")
print(f"X 和 -X 的欧几里得距离: {euclidean_dist_matrix[0, 6]:.3f} (方向相反)")
print(f"X 和 Y 的欧几里得距离: {euclidean_dist_matrix[0, 1]:.3f} (垂直向量)")

## 4. 曼哈顿距离 (Manhattan Distance)

曼哈顿距离（也称为L1距离）计算各维度差值的绝对值之和。

In [ ]:
# 计算曼哈顿距离矩阵
manhattan_dist_matrix = manhattan_distances(test_vectors)

# 可视化曼哈顿距离矩阵
plt.figure(figsize=(10, 8))
sns.heatmap(manhattan_dist_matrix, 
            xticklabels=vector_labels, 
            yticklabels=vector_labels,
            annot=True, 
            cmap='plasma_r',
            fmt='.3f')
plt.title('曼哈顿距离矩阵')
plt.tight_layout()
plt.show()

print("关键观察:")
print(f"X 和 2X 的曼哈顿距离: {manhattan_dist_matrix[0, 4]:.3f}")
print(f"X 和 -X 的曼哈顿距离: {manhattan_dist_matrix[0, 6]:.3f}")
print(f"X 和 Y 的曼哈顿距离: {manhattan_dist_matrix[0, 1]:.3f}")

## 5. 度量比较和分析

In [ ]:
# 选择一个查询向量进行比较
query_idx = 0  # 使用X向量作为查询
query_vector = test_vectors[query_idx]

print(f"查询向量: {vector_labels[query_idx]} = {query_vector}")
print("\n与其他向量的相似度/距离比较:")
print("-" * 70)
print(f"{'向量':<8} {'余弦相似度':<12} {'欧几里得距离':<12} {'曼哈顿距离':<12}")
print("-" * 70)

for i, label in enumerate(vector_labels):
    if i != query_idx:
        cosine_sim = cosine_sim_matrix[query_idx, i]
        euclidean_dist = euclidean_dist_matrix[query_idx, i]
        manhattan_dist = manhattan_dist_matrix[query_idx, i]
        
        print(f"{label:<8} {cosine_sim:<12.3f} {euclidean_dist:<12.3f} {manhattan_dist:<12.3f}")

## 6. 实际应用场景分析

In [ ]:
# 创建不同特征的向量来演示应用场景
# 场景1: 文档向量（稀疏特征）
doc_vectors = np.array([
    [1, 0, 2, 0, 3],    # 文档1: 包含词1和词3和词5
    [0, 1, 0, 2, 0],    # 文档2: 包含词2和词4
    [2, 0, 4, 0, 6],    # 文档3: 文档1的扩展版本
    [1, 1, 1, 1, 1],    # 文档4: 包含所有词
])

print("=== 文档相似度分析 ===")
print("文档向量 (词频表示):")
for i, vec in enumerate(doc_vectors):
    print(f"文档{i+1}: {vec}")

# 使用余弦相似度分析文档相似性
doc_cosine_sim = cosine_similarity(doc_vectors)
print("\n文档间余弦相似度:")
for i in range(len(doc_vectors)):
    for j in range(i+1, len(doc_vectors)):
        sim = doc_cosine_sim[i, j]
        print(f"文档{i+1} vs 文档{j+1}: {sim:.3f}")

print("\n观察: 文档1和文档3的余弦相似度很高(1.000)，因为它们有相同的词汇比例")

In [ ]:
# 场景2: 图像特征向量（密集特征）
np.random.seed(42)
image_vectors = np.array([
    [0.8, 0.2, 0.1],    # 图像1: 主要是红色
    [0.1, 0.8, 0.2],    # 图像2: 主要是绿色
    [0.2, 0.1, 0.8],    # 图像3: 主要是蓝色
    [0.5, 0.5, 0.1],    # 图像4: 红绿混合
])

print("\n=== 图像颜色相似度分析 ===")
print("图像颜色向量 (RGB比例):")
colors = ['主要红色', '主要绿色', '主要蓝色', '红绿混合']
for i, (vec, color) in enumerate(zip(image_vectors, colors)):
    print(f"图像{i+1} ({color}): {vec}")

# 比较不同度量在图像相似度上的表现
img_cosine = cosine_similarity(image_vectors)
img_euclidean = euclidean_distances(image_vectors)

print("\n使用余弦相似度:")
for i in range(len(image_vectors)):
    for j in range(i+1, len(image_vectors)):
        sim = img_cosine[i, j]
        print(f"{colors[i]} vs {colors[j]}: {sim:.3f}")

print("\n使用欧几里得距离:")
for i in range(len(image_vectors)):
    for j in range(i+1, len(image_vectors)):
        dist = img_euclidean[i, j]
        print(f"{colors[i]} vs {colors[j]}: {dist:.3f}")

## 7. 度量选择指南

In [ ]:
# 创建一个决策表
import pandas as pd

decision_data = {
    '度量方法': ['余弦相似度', '欧几里得距离', '曼哈顿距离'],
    '范围': ['[-1, 1]', '[0, +∞)', '[0, +∞)'],
    '对长度敏感': ['否', '是', '是'],
    '适用数据类型': ['稀疏向量', '密集向量', '高维稀疏'],
    '计算复杂度': ['O(d)', 'O(d)', 'O(d)'],
    '主要应用': ['文本检索', '图像匹配', '推荐系统']
}

df = pd.DataFrame(decision_data)
print("度量方法选择指南:")
print(df.to_string(index=False))

print("\n使用建议:")
print("1. 余弦相似度: 当向量长度不重要，只关心方向时（如文档相似性）")
print("2. 欧几里得距离: 当向量长度和方向都重要时（如坐标点距离）")
print("3. 曼哈顿距离: 当特征独立且有明确物理意义时（如城市街区距离）")

## 8. 性能比较

In [ ]:
# 生成大规模数据进行性能测试
import time

# 创建不同大小的数据集
sizes = [100, 500, 1000, 2000]
dimensions = [10, 50, 100]

print("不同维度和数据规模下的性能比较:")
print("-" * 80)

for dim in dimensions:
    print(f"\n维度: {dim}")
    print(f"{'数据规模':<10} {'余弦相似度':<15} {'欧几里得距离':<15} {'曼哈顿距离':<15}")
    print("-" * 60)
    
    for size in sizes:
        # 生成随机数据
        data = np.random.rand(size, dim)
        query = np.random.rand(dim)
        
        # 测试余弦相似度
        start_time = time.time()
        _ = cosine_similarity([query], data)
        cosine_time = time.time() - start_time
        
        # 测试欧几里得距离
        start_time = time.time()
        _ = euclidean_distances([query], data)
        euclidean_time = time.time() - start_time
        
        # 测试曼哈顿距离
        start_time = time.time()
        _ = manhattan_distances([query], data)
        manhattan_time = time.time() - start_time
        
        print(f"{size:<10} {cosine_time:<15.4f} {euclidean_time:<15.4f} {manhattan_time:<15.4f}")

## 9. 总结

在这个笔记本中，我们深入学习了：

### 主要相似度度量:
1. **余弦相似度**: 测量角度，忽略向量长度
2. **欧几里得距离**: 直线距离，考虑所有维度
3. **曼哈顿距离**: 各维度差值的绝对值之和

### 选择原则:
- **文档/文本**: 使用余弦相似度
- **图像/坐标**: 使用欧几里得距离
- **高维稀疏**: 考虑曼哈顿距离

### 性能考虑:
- 所有度量的时间复杂度都是O(d)
- 实际性能可能因实现和数据特征而异

下一个笔记本将探讨文本向量化技术。

# 相似度度量详解
# Similarity Metrics Deep Dive

本教程深入探讨各种相似度度量方法，包括:
- 余弦相似度 vs 点积
- 距离度量的选择
- 度量方法的适用场景
- 性能对比分析

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import time
import sys
sys.path.append('../src')

plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("✅ 库导入完成")

## 1. 相似度度量概览

In [ ]:
class SimilarityMetrics:
    """相似度度量工具类"""
    
    @staticmethod
    def cosine_similarity(v1, v2):
        """余弦相似度: 衡量向量夹角"""
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    
    @staticmethod
    def dot_product(v1, v2):
        """点积: 考虑向量大小和方向"""
        return np.dot(v1, v2)
    
    @staticmethod
    def euclidean_distance(v1, v2):
        """欧几里得距离: L2距离"""
        return np.sqrt(np.sum((v1 - v2) ** 2))
    
    @staticmethod
    def manhattan_distance(v1, v2):
        """曼哈顿距离: L1距离"""
        return np.sum(np.abs(v1 - v2))
    
    @staticmethod
    def chebyshev_distance(v1, v2):
        """切比雪夫距离: L∞距离"""
        return np.max(np.abs(v1 - v2))
    
    @staticmethod
    def hamming_distance(v1, v2):
        """汉明距离: 不同位置的数量"""
        return np.sum(v1 != v2)
    
    @staticmethod
    def jaccard_similarity(v1, v2):
        """Jaccard相似度: 适用于二进制向量"""
        intersection = np.sum(np.minimum(v1, v2))
        union = np.sum(np.maximum(v1, v2))
        return intersection / union if union > 0 else 0
    
    @staticmethod
    def pearson_correlation(v1, v2):
        """皮尔逊相关系数"""
        return np.corrcoef(v1, v2)[0, 1]

# 创建测试向量
np.random.seed(42)
v1 = np.array([1, 2, 3, 4, 5])
v2 = np.array([2, 4, 6, 8, 10])  # v1的2倍
v3 = np.array([5, 4, 3, 2, 1])   # v1的反向
v4 = np.array([1, 1, 1, 1, 1])   # 常数向量
v5 = np.random.randint(0, 2, 5)  # 二进制向量

print("测试向量:")
print(f"v1: {v1}")
print(f"v2: {v2} (v1的2倍)")
print(f"v3: {v3} (v1的反向)")
print(f"v4: {v4} (常数)")
print(f"v5: {v5} (二进制)")

## 2. 相似度度量比较

In [ ]:
# 计算所有向量对之间的相似度/距离
vectors = [v1, v2, v3, v4, v5]
vector_names = ['v1', 'v2 (2×v1)', 'v3 (反向)', 'v4 (常数)', 'v5 (二进制)']

metrics = {
    '余弦相似度': SimilarityMetrics.cosine_similarity,
    '点积': SimilarityMetrics.dot_product,
    '欧几里得距离': SimilarityMetrics.euclidean_distance,
    '曼哈顿距离': SimilarityMetrics.manhattan_distance,
    '切比雪夫距离': SimilarityMetrics.chebyshev_distance,
    'Jaccard相似度': SimilarityMetrics.jaccard_similarity
}

results = {}
for metric_name, metric_func in metrics.items():
    matrix = np.zeros((len(vectors), len(vectors)))
    for i in range(len(vectors)):
        for j in range(len(vectors)):
            try:
                if metric_name == 'Jaccard相似度':
                    # 对于Jaccard，需要二进制化
                    v_i = (vectors[i] > 0).astype(int)
                    v_j = (vectors[j] > 0).astype(int)
                    value = metric_func(v_i, v_j)
                else:
                    value = metric_func(vectors[i], vectors[j])
                matrix[i, j] = value
            except:
                matrix[i, j] = np.nan
    results[metric_name] = matrix

# 可视化结果
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, (metric_name, matrix) in enumerate(results.items()):
    if idx < len(axes):
        sns.heatmap(matrix, annot=True, fmt='.3f', 
                   xticklabels=vector_names, yticklabels=vector_names,
                   ax=axes[idx], cmap='viridis')
        axes[idx].set_title(metric_name)

plt.tight_layout()
plt.show()

## 3. 余弦相似度 vs 点积的详细分析

In [ ]:
# 创建不同长度但方向相同的向量
base_direction = np.array([1, 1, 1, 1])
scales = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
scaled_vectors = [scale * base_direction for scale in scales]

print("向量缩放实验:")
print("基础方向:", base_direction)
print("\n缩放因子 | 向量 | 长度 | 与基础向量的余弦相似度 | 点积")
print("-" * 70)

cosine_similarities = []
dot_products = []
vector_norms = []

for scale, vector in zip(scales, scaled_vectors):
    norm = np.linalg.norm(vector)
    cosine_sim = SimilarityMetrics.cosine_similarity(base_direction, vector)
    dot_prod = SimilarityMetrics.dot_product(base_direction, vector)
    
    vector_norms.append(norm)
    cosine_similarities.append(cosine_sim)
    dot_products.append(dot_prod)
    
    print(f"{scale:8.1f} | {vector} | {norm:5.2f} | {cosine_sim:16.3f} | {dot_prod:6.1f}")

# 可视化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 左图: 余弦相似度和点积 vs 缩放因子
ax1.plot(scales, cosine_similarities, 'o-', label='余弦相似度', linewidth=2)
ax1.plot(scales, np.array(dot_products)/max(dot_products), 's-', label='点积 (归一化)', linewidth=2)
ax1.set_xlabel('缩放因子')
ax1.set_ylabel('相似度值')
ax1.set_title('余弦相似度 vs 点积 (方向相同的向量)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右图: 点积 vs 向量长度
ax2.scatter(vector_norms, dot_products, s=100, alpha=0.7)
ax2.set_xlabel('向量长度')
ax2.set_ylabel('点积值')
ax2.set_title('点积与向量长度的关系')
ax2.grid(True, alpha=0.3)

# 添加趋势线
z = np.polyfit(vector_norms, dot_products, 1)
p = np.poly1d(z)
ax2.plot(vector_norms, p(vector_norms), "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

print("\n重要观察:")
print("- 余弦相似度对向量长度不敏感，只关注方向")
print("- 点积同时考虑方向和大小")
print("- 对于归一化向量，余弦相似度等于点积")

## 4. 距离度量的几何直觉

In [ ]:
# 2D示例：不同距离度量的可视化
point_a = np.array([1, 1])
point_b = np.array([4, 3])

# 计算各种距离
euclidean_dist = SimilarityMetrics.euclidean_distance(point_a, point_b)
manhattan_dist = SimilarityMetrics.manhattan_distance(point_a, point_b)
chebyshev_dist = SimilarityMetrics.chebyshev_distance(point_a, point_b)

print(f"点A: {point_a}, 点B: {point_b}")
print(f"欧几里得距离: {euclidean_dist:.3f}")
print(f"曼哈顿距离: {manhattan_dist:.3f}")
print(f"切比雪夫距离: {chebyshev_dist:.3f}")

# 可视化距离
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 欧几里得距离
axes[0].plot([point_a[0], point_b[0]], [point_a[1], point_b[1]], 'r-', linewidth=3, label='直线距离')
axes[0].plot(point_a[0], point_a[1], 'bo', markersize=10, label='点A')
axes[0].plot(point_b[0], point_b[1], 'ro', markersize=10, label='点B')
axes[0].set_title(f'欧几里得距离: {euclidean_dist:.3f}')
axes[0].grid(True, alpha=0.3)
axes[0].legend()
axes[0].set_aspect('equal')

# 曼哈顿距离
axes[1].plot([point_a[0], point_b[0]], [point_a[1], point_a[1]], 'g-', linewidth=3, label='水平移动')
axes[1].plot([point_b[0], point_b[0]], [point_a[1], point_b[1]], 'g-', linewidth=3, label='垂直移动')
axes[1].plot(point_a[0], point_a[1], 'bo', markersize=10, label='点A')
axes[1].plot(point_b[0], point_b[1], 'ro', markersize=10, label='点B')
axes[1].set_title(f'曼哈顿距离: {manhattan_dist:.3f}')
axes[1].grid(True, alpha=0.3)
axes[1].legend()
axes[1].set_aspect('equal')

# 切比雪夫距离
diff_x = abs(point_b[0] - point_a[0])
diff_y = abs(point_b[1] - point_a[1])
max_diff = max(diff_x, diff_y)

# 画正方形表示切比雪夫距离
square_x = [point_a[0] - max_diff, point_a[0] + max_diff, point_a[0] + max_diff, point_a[0] - max_diff, point_a[0] - max_diff]
square_y = [point_a[1] - max_diff, point_a[1] - max_diff, point_a[1] + max_diff, point_a[1] + max_diff, point_a[1] - max_diff]
axes[2].plot(square_x, square_y, 'purple', linewidth=2, alpha=0.7, label='切比雪夫距离边界')
axes[2].plot(point_a[0], point_a[1], 'bo', markersize=10, label='点A')
axes[2].plot(point_b[0], point_b[1], 'ro', markersize=10, label='点B')
axes[2].set_title(f'切比雪夫距离: {chebyshev_dist:.3f}')
axes[2].grid(True, alpha=0.3)
axes[2].legend()
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

## 5. 高维空间中的距离度量行为

In [ ]:
# 维度诅咒演示
dimensions = [2, 5, 10, 20, 50, 100, 200]
n_samples = 1000

euclidean_stats = []
cosine_stats = []

for dim in dimensions:
    print(f"处理 {dim} 维...")
    
    # 生成随机向量
    vectors = np.random.randn(n_samples, dim)
    
    # 计算成对距离和相似度
    euclidean_distances = []
    cosine_similarities = []
    
    # 采样以减少计算量
    sample_size = min(100, n_samples)
    sample_indices = np.random.choice(n_samples, sample_size, replace=False)
    
    for i in range(sample_size):
        for j in range(i+1, sample_size):
            v1, v2 = vectors[sample_indices[i]], vectors[sample_indices[j]]
            
            euclidean_distances.append(SimilarityMetrics.euclidean_distance(v1, v2))
            cosine_similarities.append(SimilarityMetrics.cosine_similarity(v1, v2))
    
    euclidean_stats.append({
        'mean': np.mean(euclidean_distances),
        'std': np.std(euclidean_distances),
        'min': np.min(euclidean_distances),
        'max': np.max(euclidean_distances)
    })
    
    cosine_stats.append({
        'mean': np.mean(cosine_similarities),
        'std': np.std(cosine_similarities),
        'min': np.min(cosine_similarities),
        'max': np.max(cosine_similarities)
    })

# 可视化维度效应
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 欧几里得距离统计
euclidean_means = [stats['mean'] for stats in euclidean_stats]
euclidean_stds = [stats['std'] for stats in euclidean_stats]

ax1.plot(dimensions, euclidean_means, 'o-', linewidth=2, label='均值')
ax1.fill_between(dimensions, 
                np.array(euclidean_means) - np.array(euclidean_stds),
                np.array(euclidean_means) + np.array(euclidean_stds),
                alpha=0.3, label='±1标准差')
ax1.set_xlabel('维度')
ax1.set_ylabel('欧几里得距离')
ax1.set_title('欧几里得距离 vs 维度')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 余弦相似度统计
cosine_means = [stats['mean'] for stats in cosine_stats]
cosine_stds = [stats['std'] for stats in cosine_stats]

ax2.plot(dimensions, cosine_means, 's-', linewidth=2, color='green', label='均值')
ax2.fill_between(dimensions, 
                np.array(cosine_means) - np.array(cosine_stds),
                np.array(cosine_means) + np.array(cosine_stds),
                alpha=0.3, color='green', label='±1标准差')
ax2.set_xlabel('维度')
ax2.set_ylabel('余弦相似度')
ax2.set_title('余弦相似度 vs 维度')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 距离分布的变化
euclidean_ranges = [stats['max'] - stats['min'] for stats in euclidean_stats]
cosine_ranges = [stats['max'] - stats['min'] for stats in cosine_stats]

ax3.plot(dimensions, euclidean_stds, 'o-', linewidth=2, label='欧几里得距离标准差')
ax3.plot(dimensions, cosine_stds, 's-', linewidth=2, label='余弦相似度标准差')
ax3.set_xlabel('维度')
ax3.set_ylabel('标准差')
ax3.set_title('分布离散程度 vs 维度')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 相对标准差（变异系数）
euclidean_cv = [std/mean for mean, std in zip(euclidean_means, euclidean_stds)]
cosine_cv = [std/abs(mean) if mean != 0 else 0 for mean, std in zip(cosine_means, cosine_stds)]

ax4.plot(dimensions, euclidean_cv, 'o-', linewidth=2, label='欧几里得距离CV')
ax4.plot(dimensions, cosine_cv, 's-', linewidth=2, label='余弦相似度CV')
ax4.set_xlabel('维度')
ax4.set_ylabel('变异系数 (CV)')
ax4.set_title('相对变异性 vs 维度')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n维度诅咒观察:")
print("- 高维空间中，欧几里得距离趋向于相同值")
print("- 余弦相似度在高维下表现更稳定")
print("- 距离的区分能力随维度增加而下降")

## 6. 性能基准测试

In [ ]:
# 性能测试
def benchmark_metrics(n_vectors=1000, vector_dim=128, n_queries=100):
    """基准测试不同相似度度量的性能"""
    
    # 生成测试数据
    vectors = np.random.randn(n_vectors, vector_dim).astype(np.float32)
    queries = np.random.randn(n_queries, vector_dim).astype(np.float32)
    
    # 归一化（对某些度量很重要）
    vectors_norm = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    queries_norm = queries / np.linalg.norm(queries, axis=1, keepdims=True)
    
    results = {}
    
    # 1. 余弦相似度（手动实现）
    start_time = time.time()
    for query in queries_norm:
        similarities = []
        for vector in vectors_norm:
            sim = np.dot(query, vector)  # 归一化后的点积等于余弦相似度
            similarities.append(sim)
    manual_cosine_time = time.time() - start_time
    results['手动余弦相似度'] = manual_cosine_time
    
    # 2. 向量化余弦相似度
    start_time = time.time()
    similarities_matrix = np.dot(queries_norm, vectors_norm.T)
    vectorized_cosine_time = time.time() - start_time
    results['向量化余弦相似度'] = vectorized_cosine_time
    
    # 3. sklearn余弦相似度
    start_time = time.time()
    sklearn_similarities = cosine_similarity(queries, vectors)
    sklearn_cosine_time = time.time() - start_time
    results['sklearn余弦相似度'] = sklearn_cosine_time
    
    # 4. 欧几里得距离（手动）
    start_time = time.time()
    for query in queries:
        distances = []
        for vector in vectors:
            dist = np.sqrt(np.sum((query - vector) ** 2))
            distances.append(dist)
    manual_euclidean_time = time.time() - start_time
    results['手动欧几里得距离'] = manual_euclidean_time
    
    # 5. sklearn欧几里得距离
    start_time = time.time()
    sklearn_distances = euclidean_distances(queries, vectors)
    sklearn_euclidean_time = time.time() - start_time
    results['sklearn欧几里得距离'] = sklearn_euclidean_time
    
    return results

# 执行基准测试
print("执行性能基准测试...")
perf_results = benchmark_metrics(n_vectors=1000, vector_dim=128, n_queries=100)

print("\n性能结果 (秒):")
for method, time_taken in sorted(perf_results.items(), key=lambda x: x[1]):
    print(f"{method:20s}: {time_taken:.4f}s")

# 可视化性能结果
methods = list(perf_results.keys())
times = list(perf_results.values())

plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(methods)), times, color=['red', 'green', 'blue', 'orange', 'purple'])
plt.xlabel('方法')
plt.ylabel('时间 (秒)')
plt.title('相似度度量性能比较')
plt.xticks(range(len(methods)), methods, rotation=45, ha='right')

# 添加数值标签
for bar, time_val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{time_val:.3f}s', ha='center', va='bottom')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 计算加速比
baseline_time = perf_results['手动余弦相似度']
print("\n加速比 (相对于手动余弦相似度):")
for method, time_taken in perf_results.items():
    speedup = baseline_time / time_taken
    print(f"{method:20s}: {speedup:.1f}x")

## 7. 度量方法选择指南

In [ ]:
# 创建选择指南表格
import pandas as pd

guidance_data = {
    '度量方法': [
        '余弦相似度',
        '点积',
        '欧几里得距离',
        '曼哈顿距离',
        '切比雪夫距离',
        'Jaccard相似度'
    ],
    '适用场景': [
        '文本相似度、推荐系统',
        '归一化向量的快速相似度',
        '几何距离、聚类',
        '高维稀疏数据',
        '游戏AI、路径规划',
        '集合相似度、二进制特征'
    ],
    '优点': [
        '不受向量长度影响',
        '计算快速',
        '直观的几何意义',
        '对异常值鲁棒',
        '关注最大差异',
        '适合稀疏数据'
    ],
    '缺点': [
        '计算相对复杂',
        '需要归一化',
        '维度诅咒敏感',
        '丢失方向信息',
        '只考虑单个维度',
        '仅适用于二进制数据'
    ],
    '计算复杂度': [
        'O(d)',
        'O(d)',
        'O(d)',
        'O(d)',
        'O(d)',
        'O(d)'
    ]
}

guidance_df = pd.DataFrame(guidance_data)
print("相似度度量选择指南:")
print("=" * 80)
for idx, row in guidance_df.iterrows():
    print(f"\n{row['度量方法']}:")
    print(f"  适用场景: {row['适用场景']}")
    print(f"  优点: {row['优点']}")
    print(f"  缺点: {row['缺点']}")
    print(f"  复杂度: {row['计算复杂度']}")

# 创建决策树可视化
print("\n\n度量方法选择决策树:")
print("=" * 50)
print("""
数据类型？
├── 二进制/集合 → Jaccard相似度
├── 连续数值
│   ├── 关注方向（不关注大小）→ 余弦相似度
│   ├── 关注大小和方向 → 点积（归一化向量）
│   ├── 几何距离
│   │   ├── 低维度 → 欧几里得距离
│   │   ├── 高维度/稀疏 → 曼哈顿距离
│   │   └── 最大差异重要 → 切比雪夫距离
│   └── 文本/语义相似度 → 余弦相似度
""")

print("\n实用建议:")
print("1. 文本相似度: 优先使用余弦相似度")
print("2. 推荐系统: 余弦相似度或点积")
print("3. 图像相似度: 欧几里得距离或余弦相似度")
print("4. 高维稀疏数据: 曼哈顿距离")
print("5. 性能要求高: 点积（归一化向量）")
print("6. 需要向量化计算: numpy或sklearn实现")

## 8. 总结

本教程深入探讨了各种相似度度量方法:

### 主要收获
1. **余弦相似度**: 最常用于文本和高维数据，不受向量长度影响
2. **点积**: 考虑向量大小，归一化后等于余弦相似度
3. **距离度量**: 各有特点，选择取决于数据类型和应用场景
4. **维度诅咒**: 高维空间中欧几里得距离失效，余弦相似度更稳定
5. **性能优化**: 向量化计算比循环快数十倍

### 实践建议
- 文本数据: 余弦相似度
- 图像数据: 欧几里得距离或余弦相似度
- 稀疏数据: 曼哈顿距离或Jaccard相似度
- 性能优先: 使用向量化实现

In [ ]:
print("🎉 相似度度量教程完成！")
print("📚 下一步: 学习 03_text_embeddings.ipynb")
print("💡 练习: 尝试在自己的数据上比较不同度量方法")